In [1]:
## Import necessary libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../Dataset/sp500_for_train.csv", index_col='Symbol')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499 entries, AAPL to AMTM
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Industry           499 non-null    object 
 1   Currentprice       499 non-null    float64
 2   City               499 non-null    object 
 3   Fulltimeemployees  490 non-null    float64
 4   Weight             499 non-null    float64
 5   Growth_Category    499 non-null    object 
dtypes: float64(3), object(3)
memory usage: 27.3+ KB
